# Statistical Analysis

**RankEval** provides the following statistical analysis tools: *i)* Fisher's randomization test for statistical significance, and *ii)* bias/variance decomposition of the error.

In [1]:
# import common libraries
%load_ext autoreload
%autoreload 2

## Statistical Significance

According to the work by *M.D. Smucker, J. Allan, B. Carterette, "A Comparison of Statistical Significance Tests for Information Retrieval Evaluation", CIKM 2007*, **Fisher's randomization test** is the most appropriate statistical test to evaluate wheter two rankers differ significantly.

We first shortly describe the test. The null hypthesis is that the two given rankers A and B are indentical: an underlying ranker R is asked to produce two rankings for each given query  and these two rankings are randomly labeled as ranker A or ranker B. The goal of the test is to measure the probability that the observed performance gap between ranker A and B is due to a random labeling.

Under the null hypthesis, every permutation of the labelling is equally probable. If we enumerate all the possible A-B labelings, and we measure the corresponding quality gap, we have that:
 - the *one-sided p-value* is given by the fraction of times the quality difference is larger than the originally observed difference;
 - the *two-sided p-value* is given by the fraction of times the resulting quality *absolute difference* is larger than the originally observed difference.

Since the number of permutations is exponential in the number of queries, a large number of random permutations is used.

##### Import RankEval statistical significance tools

In [2]:
from rankeval.model import RTEnsemble
from rankeval.dataset import Dataset
from rankeval.dataset.datasets_fetcher import load_dataset
from rankeval.metrics import NDCG
from rankeval.metrics import Precision
from rankeval.analysis.statistical import statistical_significance

##### Load models and data from file

In [3]:
# load data
dataset_container = load_dataset(dataset_name='msn10k', 
                                fold='1', 
                                download_if_missing=True, 
                                force_download=False, 
                                with_models=True)

# we use the test dataset here
msn1 = dataset_container.test_dataset
msn1.name = "MSN10K - Fold 1"

Loading files. This may take a few minutes.
done loading dataset!


In [4]:
# view available models
for item, file_name in enumerate(dataset_container.model_filenames):
    print item, file_name

0 /home/rankeval/rankeval_data/msn10k/models/Fold1/lightgbm/LGBM.msn10k.fold-1.lambdarank.leaves-32.lr-5.trees-1000.model
1 /home/rankeval/rankeval_data/msn10k/models/Fold1/catboost/msn1.catboost.LAMBDAMART.1000.5.T1000.json
2 /home/rankeval/rankeval_data/msn10k/models/Fold1/catboost/msn1.catboost.LAMBDAMART.1000.5.T1000.model
3 /home/rankeval/rankeval_data/msn10k/models/Fold1/quickrank/msn1.quickrank.LAMBDAMART.20000.32.T20000.xml
4 /home/rankeval/rankeval_data/msn10k/models/Fold1/quickrank/msn1.quickrank.LAMBDAMART.20000.32.T5000.xml
5 /home/rankeval/rankeval_data/msn10k/models/Fold1/quickrank/msn1.quickrank.LAMBDAMART.20000.32.T1000.xml
6 /home/rankeval/rankeval_data/msn10k/models/Fold1/quickrank/msn1.quickrank.LAMBDAMART.20000.32.T15000.xml
7 /home/rankeval/rankeval_data/msn10k/models/Fold1/quickrank/msn1.quickrank.LAMBDAMART.20000.32.T10000.xml
8 /home/rankeval/rankeval_data/msn10k/models/Fold1/xgboost/XGBOOST.msn10k.fold-1.pairwise.d-5.lr-10.trees-1000.model
9 /home/rankeval/rank

In [5]:
# model files
qr_10K_file  = dataset_container.model_filenames[7] # "msn1.quickrank.LAMBDAMART.20000.32.T10000.xml"
qr_1K_file   = dataset_container.model_filenames[5] # "msn1.quickrank.LAMBDAMART.20000.32.T1000.xml"
lgbm_1K_file = dataset_container.model_filenames[0] # "LGBM.msn10k.fold-1.lambdarank.leaves-32.lr-5.trees-1000.model"
xbg_1k_file  = dataset_container.model_filenames[8] # "XGBOOST.msn10k.fold-1.pairwise.d-5.lr-10.trees-1000.model"

qr_1K   = RTEnsemble(qr_1K_file, name="QuickRank.1k", format="QuickRank")
qr_10K  = RTEnsemble(qr_10K_file, name="QuickRank.10k", format="QuickRank")
lgbm_1K = RTEnsemble(lgbm_1K_file, name="LGBM.1k", format="LightGBM")
xgb_1K = RTEnsemble(xbg_1k_file, name="XGB.1k", format="XGBoost")

##### Run the Fisher's Randomization test

The `statistical_significance` test between a two rankers can be run on a list of datasets and for a list of IR quality metrics. The function returns both the one-sided and two-sided p-values.

We first compare the three models we loaded above. We can observe below that the QuickRank model with 10k trees performs worse that the QuickRank with 1k tree: this is due to the overfitting of such a large model. Also, LightGBM is the worst performing alorithm.

In [6]:
from rankeval.analysis.effectiveness import model_performance

ndcg_10 = NDCG(cutoff=10)

perf = model_performance(datasets=[msn1], 
                         models=[qr_1K, qr_10K, lgbm_1K, xgb_1K], 
                         metrics=[ndcg_10])
perf.to_dataframe()

Model Performance
dataset         model         metric                    
MSN10K - Fold 1 QuickRank.1k  NDCG@10           0.492246
                QuickRank.10k NDCG@10           0.474035
                LGBM.1k       NDCG@10           0.507996
                XGB.1k        NDCG@10           0.487309

We also observe that the QuickRank.1k model performs better than XGBoost.1k with only a small difference. We therefore measure whether this difference is statistically significant as follows.

In [7]:
stat_sig = statistical_significance(datasets=[msn1],
                                    model_a=qr_1K, model_b=xgb_1K, 
                                    metrics=[ndcg_10],
                                    n_perm=100000 )
stat_sig.to_dataframe()

Statistical Significance
dataset         metric  p-value                            
MSN10K - Fold 1 NDCG@10 one-sided                   0.03412
                        two-sided                   0.06830

We conclude that the difference is statistically significant at $p<0.05$ only for the one-sided test. To conclude the analysis, we evaluate the performance of the two algorithms also with NDCG@50 and Precision@10.

In [8]:
ndcg_50 = NDCG(cutoff=50)
prec_10 = Precision(cutoff=10)

perf = model_performance(datasets=[msn1], 
                         models=[qr_1K, xgb_1K], 
                         metrics=[ndcg_10, ndcg_50, prec_10])
perf.to_dataframe()

Model Performance
dataset         model        metric                    
MSN10K - Fold 1 QuickRank.1k NDCG@10           0.492246
                             NDCG@50           0.597562
                             P@10              0.657644
                XGB.1k       NDCG@10           0.487309
                             NDCG@50           0.595671
                             P@10              0.682194

In [9]:
stat_sig = statistical_significance(datasets=[msn1],
                                    model_a=qr_1K, model_b=xgb_1K, 
                                    metrics=[ndcg_10, ndcg_50, prec_10],
                                    n_perm=100000 )
stat_sig.to_dataframe()

Statistical Significance
dataset         metric  p-value                            
MSN10K - Fold 1 NDCG@10 one-sided                   0.03474
                        two-sided                   0.06961
                NDCG@50 one-sided                   0.14742
                        two-sided                   0.29608
                P@10    one-sided                   0.00000
                        two-sided                   0.00000

According to the above experiments, we see no sigificant differences in terms of NDCG@50, but the preformance of XGBoost is better and statistically significant in terms of Precision@10.

# Bias vs. Variance decomposition

The Error of a Algorithm can be decomposed in:
$$E(A) = Bias(A) + Variance(A) + Noise(A)$$
where:
 - Bias is how far is the model from the prediction
 - Variance is how sensitive (how changes) the prediction with different training sets (overfitting)
 - Noise is the irreducible error in the dataset (learner independent)

**RankEval** supports the computation of the bias vs. variance decomposition of the error.
The approach used is based on the works of [Webb05] and [Dom05]. As in other works, we hereinafter assume noise is absent.

RankEval allows to decompose the errore according to a given user provided (IR) quality metric as follows.

Each instance of the dataset is scored *L* times.
A single scoring is achieved by splitting the dataset at random into
*k* folds. Each fold is scored by the model *M* trained with the algorithm $A$ on the remainder folds.
[Webb05] recommends the use of 2 folds.

If the metric used is Mean Squared Error then the standard decomposition is used.
The Bias for and instance *x* is defined as mean squared error of the *L* trained models
w.r.t. the true label *y*, denoted with ${\sf E}_{L} [M(x) - y]^2$. 
The Variance for an instance *x* is measured across the *L* trained models: 
${\sf E}_{L} [M(x) - {\sf E}_{L} M(x)]^2$. 
Both are averaged over all instances in the dataset.

If the metric is any of the IR quality measures, we resort to the bias variance
decomposition of the mean squared error of the given metric w.r.t. its ideal value,
e.g., for the case of NDCG, ${\sf E}_{L} [1 - {\sf NDCG}]^2$. 
Recall that, a formal Bias/Variance decomposition was not proposed yet.

##### References
 - [Webb05] Webb, Geoffrey I., and Paul Conilione. "Estimating bias and variance from data." Pre-publication manuscript (2005).
 - [Dom05] Domingos P. A. "Unified bias-variance decomposition." In Proceedings of 17th International Conference on Machine Learning 2000 (pp. 231-238).

##### Load dataset and define metrics of interest

In [10]:
from rankeval.analysis.statistical import bias_variance

from rankeval.dataset import Dataset
from rankeval.dataset.datasets_fetcher import load_dataset
from rankeval.metrics import NDCG
from rankeval.metrics import MSE

In [11]:
# load data
dataset_container = load_dataset(dataset_name='msn10k', 
                                fold='1', 
                                download_if_missing=True, 
                                force_download=False, 
                                with_models=True)

# we use the test dataset here
msn1 = dataset_container.test_dataset
msn1.name = "MSN10K - Fold 1"

Loading files. This may take a few minutes.
done loading dataset!


##### Define the algorithm of wich we want to measure its bias/variance decomposition

The Bias/Variancs decomposition is a measure of a given algorithm with given parameters. Recall that RankEval needs to repeatedly train and evaluate models learnt by the given algorithm. To do so, we define a wrapper function to be used by RankEval with the following parameters:
 - `train_X`: numpy.ndarray storing a 2-D matrix of size num_docs x num_features
 - `train_Y`: numpy.ndarray storing a vector of document's relevance labels
 - `train_q`: numpy.ndarray storing a vector of query lengths
 - `test_X`: numpy.ndarray as for `train_X`
Such wrapper function trains a new model on `train_X`, `train_Y`, `train_q`, then used to score `test_X`.
An `numpy.ndarray` with such scores is returned.

In the example below we use LightGBM, for which we define a two wrapper function for training forests of 100 trees and with eithr 32 (`lgbm_small_wrapper`) or 64 (`lgbm_large_wrapper`) leaves each.

In [12]:
import lightgbm

def lgbm_algo(trees, leaves, train_X, train_Y, train_q, test_X):
    params = {'num_leaves': leaves, 'objective':'lambdarank', 'learning_rate': 0.05}

    training = lightgbm.Dataset(data=train_X, label=train_Y, group=train_q)
    
    bst = lightgbm.train(params, training, num_boost_round=trees)
    
    return bst.predict(test_X)

def lgbm_small(train_X, train_Y, train_q, test_X):
    return lgbm_algo(100, 16, train_X, train_Y, train_q, test_X)

def lgbm_large(train_X, train_Y, train_q, test_X):
    return lgbm_algo(100, 128, train_X, train_Y, train_q, test_X)

##### Run the bias/variance decomposition

The function `bias_variance` returns an `array.DataArray` containing the bias/variance decomposition of the error for the given datasets, algorithms and metrics.

Below the bias variance decomposition for the two LightGBM models for both the MSE and NDCG@10 losses. 

The error is dominated by the Bias component. Interestingly, the variance increases by a factor of 2 with the larger model including 128 leaves. Also note that the smaller model is the best performing in terms of MSE but not in terms of NDCG. This is because the model was trained by optimizing the Lambda-Rank loss and not the MSE: predicted scores may deviate from the target labels, but the resulting ranking is improved.

In [13]:
bv = bias_variance(datasets=[msn1], 
                   algos=[lgbm_small, lgbm_large], 
                   metrics=[MSE(), NDCG(cutoff=10)], 
                   L=5, k=2)
bv.to_dataframe()

Bias/Variance Decomposition
dataset         metric  algo       error                                
MSN10K - Fold 1 MSE     lgbm_small Error                        1.286307
                                   Bias                         1.284281
                                   Variance                     0.002025
                        lgbm_large Error                        1.421161
                                   Bias                         1.416336
                                   Variance                     0.004825
                NDCG@10 lgbm_small Error                        0.344878
                                   Bias                         0.341951
                                   Variance                     0.002927
                        lgbm_large Error                        0.329454
                                   Bias                         0.324732
                                   Variance                     0.004721